 id3 

In [46]:
class TreeNode:
    def __init__(self,feature=None,entropy=None,samples=None,value=None,left=None,right=None) -> None:
        self.feature = feature
        self.entropy = entropy
        self.samples = samples
        self.value = value
        self.left = left
        self.right = right
    
    def is_leaf(self):
        return self.left is None and self.right is None
    
def compute_entropy(y):
    entropy = 0
    if len(y) > 0:
        p1 = len(y[y == 1])/len(y)
        if p1 not in (0,1):
            entropy = -p1*np.log2(p1) -(1-p1)*np.log2(1-p1)
    return entropy

def split_dataset(x,node_indices,feature):
    left_indices,right_indices = [],[]
    for i in node_indices:
        if x[i][feature] == 1:
            left_indices.append(i)
        else:
            right_indices.append(i)
    return left_indices,right_indices

def compute_information_gain(x,y,node_indices,feature):
    left_indices,right_indices = split_dataset(x,node_indices,feature)
    x_node,y_node = x[node_indices],y[node_indices] 
    x_left,y_left = x[left_indices],y[left_indices]
    x_right,y_right = x[right_indices],y[right_indices]
    node_entropy = compute_entropy(y_node)
    left_entropy = compute_entropy(y_left)  
    right_entropy = compute_entropy(y_right)
    w_left = len(x_left)/len(x)
    w_right = len(x_right)/len(x)
    weighted_entropy = w_left*left_entropy + w_right*right_entropy
    information_gain = node_entropy - weighted_entropy
    return information_gain

def get_best_split(x,y,node_indices):
    num_features = x.shape[1]
    max_info_gain = -np.inf
    best_feature = None
    for feature in range(num_features):
        info_gain = compute_information_gain(x,y,node_indices,feature)
        if info_gain > max_info_gain:
            max_info_gain = info_gain 
            best_feature = feature
    return best_feature

def build_id3_tree(x,y,node_indices,current_depth,max_depth):
    entropy = compute_entropy(y[node_indices])
    value = np.bincount(y[node_indices],minlength=2)
    node = TreeNode(entropy=entropy,samples=len(node_indices),value=value)
    #Stop condition
    if current_depth == max_depth or entropy == 0 or len(np.unique(y[node_indices]))==1:
        return node
    best_feature = get_best_split(x,y,node_indices)
    if best_feature is None:
        return node
    left_indices,right_indices = split_dataset(x,node_indices,best_feature)
    node.feature = best_feature
    if left_indices:
        node.left = build_id3_tree(x,y,left_indices,current_depth+1,max_depth)
    if right_indices:
        node.right = build_id3_tree(x,y,right_indices,current_depth+1,max_depth)
    return node

def print_tree(node, depth=0, feature_names=None):
    # Determine the feature name based on whether a list of feature names was provided.
    # If the node is not a leaf and feature names are provided, use the actual feature name.
    # Otherwise, use a generic feature label.
    if feature_names is not None and node.feature is not None:
        feature_name = feature_names[node.feature]
    else:
        feature_name = f"Feature {node.feature}"

    # Check if the current node is a leaf node.
    if node.is_leaf():
        # For leaf nodes, print the class distribution of the samples.
        print(f"{'  ' * depth}Leaf node, Class distribution: {node.value}")
    else:
        # For internal nodes, print the feature name, entropy, and number of samples.
        print(f"{'  ' * depth}{feature_name} (entropy={node.entropy:.3f}, samples={node.samples})")

        # If there is a left child, print the left branch of the tree.
        if node.left is not None:
            print(f"{'  ' * depth}Left:")
            print_tree(node.left, depth + 1, feature_names)

        # If there is a right child, print the right branch of the tree.
        if node.right is not None:
            print(f"{'  ' * depth}Right:")
            print_tree(node.right, depth + 1, feature_names)

# Correct function name for building the tree
root_node = build_id3_tree(x, y, list(range(len(y))), current_depth=0, max_depth=max_depth)

# Print the tree
# Assuming df is defined elsewhere and is a pandas DataFrame
feature_names = df.drop('Decision', axis=1).columns.tolist()
print_tree(root_node, feature_names=feature_names)


def predict(tree, sample, feature_names):
    """
    Predict the class label for a single sample based on the decision tree.
    
    :param tree: The root node of the decision tree.
    :param sample: The feature vector for the sample to predict.
    :param feature_names: The list of feature names corresponding to the indices in the sample.
    :return: The predicted class label.
    """
    # Traverse the tree until a leaf node is reached
    while not tree.is_leaf():
        # If the feature on which to split is in our sample, use it; otherwise, predict randomly.
        if tree.feature is not None and feature_names[tree.feature] in sample:
            # Determine the index of the feature to split on
            feature_index = feature_names.index(tree.feature)
            # If the feature of the sample is 1, go right; otherwise, go left.
            if sample[feature_index] == 1:
                tree = tree.right
            else:
                tree = tree.left
        else:
            # If the feature is not in our sample, we have an issue (e.g., missing feature)
            # For simplicity, return a random class label (0 or 1).
            return np.random.choice([0, 1])
    
    # Once a leaf node is reached, return the class with the highest count
    return np.argmax(tree.value)

my_sample = {
    'Outlook_Overcast': 0,
    'Outlook_Rain': 1,
    'Outlook_Sunny': 0,
    'Temp': 0,  # Assuming this is a binary feature after preprocessing
    'Humidity': 1,  # Assuming this is a binary feature after preprocessing
    'Wind': 0  # Assuming this is a binary feature after preprocessing
}

# Convert the sample dictionary to a list in the order of feature names used in the tree
sample_features = [my_sample[fn] for fn in feature_names]

# Use the predict function to get the prediction for the sample
predicted_class = predict(root_node, sample_features, feature_names)
print(f"The predicted class for the sample is: {predicted_class}")

Outlook_Overcast (entropy=0.940, samples=14)
Left:
  Leaf node, Class distribution: [0 4]
Right:
  Humidity (entropy=1.000, samples=10)
  Left:
    Outlook_Rain (entropy=0.985, samples=7)
    Left:
      Leaf node, Class distribution: [1 3]
    Right:
      Leaf node, Class distribution: [3 0]
  Right:
    Outlook_Rain (entropy=0.918, samples=3)
    Left:
      Leaf node, Class distribution: [1 0]
    Right:
      Leaf node, Class distribution: [0 2]
The predicted class for the sample is: 1
